In [1]:
!git clone https://github.com/Asikpalysik/Automatic-License-Plate-Detection.git

Cloning into 'Automatic-License-Plate-Detection'...
remote: Enumerating objects: 780, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 780 (delta 79), reused 84 (delta 76), pack-reused 693 (from 1)
Receiving objects: 100% (780/780), 97.96 MiB | 32.79 MiB/s, done.
Resolving deltas: 100% (254/254), done.
Updating files: 100% (939/939), done.


In [2]:
import shutil
shutil.move("/content/Automatic-License-Plate-Detection/images", "/content/")

'/content/images'

In [27]:
!pip install pytesseract
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet

from glob import glob
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2, VGG16, EfficientNetB7, ResNet50
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [28]:
path = glob('/content/images/*.xml')
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])

for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

In [29]:
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,/content/images/N19.xml,99,216,367,423
1,/content/images/N242.xml,202,386,190,227
2,/content/images/N167.xml,345,848,242,416
3,/content/images/N184.xml,229,1391,204,660
4,/content/images/N103.xml,813,1067,665,724


In [30]:
filename = df['filepath'][0]
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('/content/images',filename_image)
    return filepath_image
getFilename(filename)

'/content/images/N19.jpeg'

In [31]:
image_path = list(df['filepath'].apply(getFilename))
image_path[:10]#random check

['/content/images/N19.jpeg',
 '/content/images/N242.jpeg',
 '/content/images/N167.jpeg',
 '/content/images/N184.jpeg',
 '/content/images/N103.jpeg',
 '/content/images/N59.jpeg',
 '/content/images/N239.jpeg',
 '/content/images/N174.jpeg',
 '/content/images/N203.jpeg',
 '/content/images/N122.jpeg']

In [32]:
# prompt: file_path = image_path[87] #path of our image N2.jpeg
# img = cv2.imread(file_path) #read the image
# # xmin-1804/ymin-1734/xmax-2493/ymax-1882
# img = io.imread(file_path) #Read the image
# fig = px.imshow(img)
# fig.update_layout(width=600, height=500, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure 8 - N2.jpeg with bounding box')
# fig.add_shape(type='rect',x0=1804, x1=2493, y0=1734, y1=1882, xref='x', yref='y',line_color='cyan')
# Take the image and find its xml file and then use that xml file to create the bbox on image as above

import plotly.express as px
import xml.etree.ElementTree as xet
import os
from skimage import io

# Assuming df and image_path are defined as in the previous code

file_path = image_path[81]  # path of our image N2.jpeg

# Extract bounding box coordinates from the corresponding XML file
filename = df['filepath'][81]  # Get the XML filename corresponding to the image
info = xet.parse(filename)
root = info.getroot()
member_object = root.find('object')
labels_info = member_object.find('bndbox')
xmin = int(labels_info.find('xmin').text)
xmax = int(labels_info.find('xmax').text)
ymin = int(labels_info.find('ymin').text)
ymax = int(labels_info.find('ymax').text)


img = io.imread(file_path)  # Read the image
fig = px.imshow(img)
fig.update_layout(
    width=600,
    height=500,
    margin=dict(l=10, r=10, b=10, t=10),
    xaxis_title="Figure 8 - N2.jpeg with bounding box",
)
fig.add_shape(
    type="rect",
    x0=xmin,
    x1=xmax,
    y0=ymin,
    y1=ymax,
    xref="x",
    yref="y",
    line_color="cyan",
)
fig.show()


Output hidden; open in https://colab.research.google.com to view.

In [33]:
#Targeting all our values in array selecting all columns
labels = df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # Prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output
    # Append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [34]:
# Convert data to array
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [35]:
# Split the data into training and testing set using sklearn.
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

In [63]:
resnet = ResNet50(weights="imagenet",include_top=False, input_tensor=Input(shape=(224,224,3)))
# ---------------------
headmodel = resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)

# ---------- model
model = Model(inputs=resnet.input,outputs=headmodel)

In [64]:
# Complie model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 73,890,466 (281.87 MB)

 Trainable params: 73,837,346 (281.67 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [41]:
tfb = TensorBoard('object_detection')
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=50,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 197s 2s/step - loss: 0.0815 - val_loss: 0.0667
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 31s 229ms/step - loss: 0.0609 - val_loss: 0.0713
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 247ms/step - loss: 0.0504 - val_loss: 0.0730
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 232ms/step - loss: 0.0459 - val_loss: 0.0647
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 232ms/step - loss: 0.0389 - val_loss: 0.0493
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 244ms/step - loss: 0.0124 - val_loss: 0.0217
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 231ms/step - loss: 0.0061 - val_loss: 0.0151
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 236ms/step - loss: 0.0057 - val_loss: 0.0144
Epoch 9/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 236ms/step - loss: 0.0031 - val_loss: 0.0101
Epoch 10/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 231ms/step - loss: 0.0025 - val_loss: 0.0102
Epoch 11/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 232ms/step - loss: 0.0018 - val_loss: 0.0089
Epoch 12/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 231ms/step

In [15]:
model.save('./object_detection.keras')

In [65]:
path = '/content/images/N6.jpeg'
image = load_img(path) # PIL object
image = np.array(image,dtype=np.uint8) # 8 bit array (0,255)
image1 = load_img(path,target_size=(224,224))
image_arr_224 = img_to_array(image1)/255.0  # Convert into array and get the normalized output

# Size of the orginal image
h,w,d = image.shape
print('Height of the image =',h)
print('Width of the image =',w)

Height of the image = 1080
Width of the image = 1920


In [66]:
fig = px.imshow(image)
fig.update_layout(width=700, height=500,  margin=dict(l=10, r=10, b=10, t=10), xaxis_title='Figure 13 - TEST Image')

Output hidden; open in https://colab.research.google.com to view.

In [67]:
image_arr_224.shape

(224, 224, 3)

In [68]:
test_arr = image_arr_224.reshape(1,224,224,3)
test_arr.shape

(1, 224, 224, 3)

In [69]:
# Make predictions
coords = model.predict(test_arr)
coords

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


array([[0.5125358 , 0.804397  , 0.42053792, 0.477338  ]], dtype=float32)

In [70]:
# Denormalize the values
denorm = np.array([w,w,h,h])
coords = coords * denorm
coords

array([[ 984.0687561 , 1544.44221497,  454.18095231,  515.52502513]])

In [71]:
coords = coords.astype(np.int32)
coords

array([[ 984, 1544,  454,  515]], dtype=int32)

In [72]:
# Draw bounding on top the image
xmin, xmax,ymin,ymax = coords[0]
pt1 =(xmin,ymin)
pt2 =(xmax,ymax)
print(pt1, pt2)

(984, 454) (1544, 515)


In [73]:
cv2.rectangle(image,pt1,pt2,(0,255,0),3)
fig = px.imshow(image)
fig.update_layout(width=700, height=500, margin=dict(l=10, r=10, b=10, t=10))

Output hidden; open in https://colab.research.google.com to view.

In [74]:
# Create pipeline
path = '/content/images/N6.jpeg'
def object_detection(path):

    # Read image
    image = load_img(path) # PIL object
    image = np.array(image,dtype=np.uint8) # 8 bit array (0,255)
    image1 = load_img(path,target_size=(224,224))

    # Data preprocessing
    image_arr_224 = img_to_array(image1)/255.0 # Convert to array & normalized
    h,w,d = image.shape
    test_arr = image_arr_224.reshape(1,224,224,3)

    # Make predictions
    coords = model.predict(test_arr)

    # Denormalize the values
    denorm = np.array([w,w,h,h])
    coords = coords * denorm
    coords = coords.astype(np.int32)

    # Draw bounding on top the image
    xmin, xmax,ymin,ymax = coords[0]
    pt1 =(xmin,ymin)
    pt2 =(xmax,ymax)
    print(pt1, pt2)
    cv2.rectangle(image,pt1,pt2,(0,255,0),3)
    return image, coords

image, cods = object_detection(path)

fig = px.imshow(image)
fig.update_layout(width=700, height=500, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure 14')

Output hidden; open in https://colab.research.google.com to view.

In [62]:
img = np.array(load_img(path))
xmin ,xmax,ymin,ymax = cods[0]
roi = img[ymin:ymax,xmin:xmax]
fig = px.imshow(roi)
fig.update_layout(width=350, height=250, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure 15 Cropped image')

In [76]:
!pip install tensorflow matplotlib tensorflow_hub tf_slim

In [78]:
!pip install tensorflow==2.8.0 matplotlib pytesseract opencv-python
!apt-get install -y tesseract-ocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 383.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 45.2 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation:

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,642 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123632 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [1]:
!cd models/research && protoc object_detection/protos/*.proto --python_out=

: No such file or directory